In [1]:
import pandas as pd
import numpy as np
from elements import elements
import elements.elements as elements
from astropy.table import Table

In [2]:
vdat = pd.read_csv('vald_test2', header=None, skiprows=2, nrows=400000)
vdat = vdat.drop(13,axis=1)
vdat = vdat[vdat[12].notna()]
vdat = vdat[1:100]
vdat = vdat.reset_index(drop=True)

/home/katrine/anaconda3/envs/carsus/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
el_list = pd.read_csv('elements.txt',delimiter=' ',header=None)
values = el_list[0]

numbers = range(len(el_list[0])+1)
el_dict = dict(zip(el_list[0], numbers[1:]))

In [4]:
wave = (vdat[1].astype(float)/10).round(4)
log_gf = (vdat[2].astype(float)).round(3)

el = vdat[0].str.split(" ", n = 1, expand = True)
el[0] = el[0].replace({'\'':''}, regex=True)

for i in range(len(el[0])):
    e = el[0][i]
    el[0][i] = el_dict[e]

el[1] = (el[1].replace({'\'':''}, regex=True).astype(int) - 1).apply(lambda x: '{0:0>2}'.format(x))
#el[1] = el[1].apply(lambda x: '{0:0>2}'.format(x))
elm = el[0].map(str) + '.' + el[1].map(str)

E_l = vdat[3].astype(float)
j_l = vdat[4].apply(lambda x: '{0:0>2}'.format(x))
label_l = pd.DataFrame(['2D' for s in range(len(vdat))])
E_u = vdat[5].astype(float)
j_u = vdat[6].apply(lambda x: '{0:0>2}'.format(x))
label_u = pd.DataFrame(['2D' for s in range(len(vdat))])
lande_l = vdat[7]*1000
lande_u = vdat[8]*1000
rad = vdat[10]
stark = vdat[11]
waals = vdat[12]
zeroes = pd.DataFrame(np.zeros((len(vdat), 1)))


df_vald = pd.concat([wave, log_gf, elm, E_l, j_l, label_l, E_u, j_u, label_u, rad, stark, waals, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, lande_l, lande_u], axis=1)
df_vald

,1,2,0,3,4,0,5,6,0,10,...,0,0,0,0,0,0,0,0,7,8
0,50.0016,-4.145,28.02,0.2814,2.0,2D,25.0774,1.0,2D,8.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,670.0,140.0
1,50.0022,-2.220,27.03,3.6690,4.0,2D,28.4647,5.0,2D,9.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1050.0,1000.0
2,50.0024,-4.340,12.03,74.0267,1.5,2D,98.8223,0.5,2D,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
3,50.0035,-4.775,20.01,11.0611,0.5,2D,35.8562,0.5,2D,9.13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1560.0,2000.0
4,50.0044,-2.532,25.04,7.7928,1.5,2D,32.5875,2.5,2D,9.64,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800.0,1230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,50.1106,-2.904,26.03,3.9979,5.5,2D,28.7400,4.5,2D,9.64,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1270.0,1110.0
95,50.1110,-0.818,27.03,4.5481,4.0,2D,29.2901,3.0,2D,9.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,1000.0
96,50.1118,-1.203,27.03,0.1683,1.0,2D,24.9098,2.0,2D,9.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,1010.0
97,50.1136,-2.664,27.03,3.7031,3.0,2D,28.4437,4.0,2D,9.48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,760.0,1220.0


In [5]:
df_vald.to_csv('VALD_linelist.dat', header=None, index=None)

In [6]:
pd.options.display.max_rows = len(df_vald)
pd.options.display.max_columns = df_vald.shape[1]
pd.set_option('display.expand_frame_repr', False)

with open('VALD_linelist.dat', 'w') as f: 
    df_vald.to_string(f, col_space=10, index=False, header=None)